In [1]:
%cd ../..

/home/eli/AnacondaProjects/combinators


In [2]:
import logging

import numpy as np
import probtorch
import torch

from combinators.inference import importance, mcmc
from combinators.kernel import kernel, mh
import combinators.model as model
from combinators import utils

In [3]:
import examples.ais.ais_1d_gaussians as ais_example
import examples.ais.image_density as image_density

In [4]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [5]:
from scipy.misc import imread
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt

img_ary = imread('examples/ais/probtorch-logo-bw.png', mode='L')
img_ary = gaussian_filter(img_ary, sigma=1)

In [6]:
ANNEALING_STEPS = 5

In [7]:
BATCH_SHAPE = (100,)

In [8]:
proposal = image_density.AnnealingProposal(image_density.ImageProposal(batch_shape=BATCH_SHAPE), ANNEALING_STEPS)

In [9]:
target = model.Compose(image_density.ProbtorchLogoDensity(batch_shape=BATCH_SHAPE), proposal)

In [10]:
ais = ais_example.annealed_importance(target, kernel.GaussianKernel('ImageProposal', 'ImageProposal'), moves=ANNEALING_STEPS)

In [11]:
annealed_points, trace, log_weight = ais(data={'image': torch.Tensor(img_ary)})

RuntimeError: The size of tensor a (100) must match the size of tensor b (150) at non-singleton dimension 1

In [ ]:
target = ais_example.AnnealingTarget(annealing_steps=5, batch_shape=(100,))

In [ ]:
ais = ais_example.annealed_importance(target, kernel.GaussianKernel('AnnealingTarget', 'X_0', scale=1.0), moves=5)

In [ ]:
annealed_points, trace, log_weight = ais()
log_probs = utils.normalize_weights(log_weight)

In [ ]:
import matplotlib.pyplot as plt
from torch.distributions import Normal

In [ ]:
sample = torch.linspace(-10, 10, 100)
sample_probs = Normal(loc=torch.zeros(1), scale=torch.ones(1)).log_prob(sample).exp()

In [ ]:
plt.scatter(annealed_points.detach().numpy(), log_weight.exp())
plt.plot(sample.detach().numpy(), sample_probs.detach().numpy())

In [ ]:
annealed_points

In [ ]:
log_probs.exp()

In [ ]:
importance_points, _, resampled_weights = importance.importance(target)(t=4)
resampled_probs = utils.normalize_weights(resampled_weights)

In [ ]:
plt.scatter(importance_points.detach().numpy(), resampled_weights.exp())
plt.plot(sample.detach().numpy(), sample_probs.detach().numpy())